In [21]:
import os
import sys

In [4]:
os.chdir("../")

In [6]:
os.getcwd()

'd:\\Dropbox\\Self-Development\\Coding_Projects\\TextSummarizer'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [60]:
import urllib.request as request
import zipfile
import patoolib
from src.constants import *
from src.utils.exception import CustomException
from src.utils.logger import logger
from src.utils.utils import load_yaml, create_directories, get_size

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = load_yaml(config_filepath)
        self.params = load_yaml(params_filepath)

        create_directories([self.config['artifacts_root']])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']

        create_directories([config['root_dir']])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_URL=config['source_URL'],
            local_data_file=config['local_data_file'],
            unzip_dir=config['unzip_dir']
        )

        return data_ingestion_config

In [61]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [62]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e, sys)

[2023-08-04 20:22:10,408: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-08-04 20:22:10,410: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-08-04 20:22:10,412: INFO: utils: created directory at: artifacts]
[2023-08-04 20:22:10,413: INFO: utils: created directory at: artifacts/data_ingestion]
[2023-08-04 20:22:10,414: INFO: 965472011: File already exists of size: ~ 2875 KB]


CustomException: Error occurred in script name [C:\Users\jinou\AppData\Local\Temp\ipykernel_95636\4056456602.py] line number [6] error message [Error occurred in script name [C:\Users\jinou\AppData\Local\Temp\ipykernel_95636\965472011.py] line number [28] error message ['>=' not supported between instances of 'str' and 'int']]